In [1]:
import math
import time
import cv2
import numpy as np
import mediapipe as mp
from cvzone.HandTrackingModule import HandDetector  

In [ ]:
cap = cv2.VideoCapture(0)
detector = HandDetector(maxHands=1)

offsets = 20
imgSize = 300

folder = "Data/A"
counter = 0

while True:
    success, img = cap.read()
    hands, img = detector.findHands(img)
    
    if hands:
        hand = hands[0]
        x, y, w, h = hand['bbox']
        
        # Ensure that the bounding box and offsets are within the image boundaries
        x1 = max(x - offsets, 0)
        y1 = max(y - offsets, 0)
        x2 = min(x + w + offsets, img.shape[1])
        y2 = min(y + h + offsets, img.shape[0])
        
        imgWhite = np.ones((imgSize , imgSize , 3) , np.uint8)*255
        # Crop the image with valid dimensions
        imgCrop = img[y1:y2, x1:x2]
        
        #setting height and width of white image and input on top
        aspectRatio = h/w
        
        if aspectRatio > 1:
            k = imgSize/h
            wCal = math.ceil(k*w)
            imgResize = cv2.resize(imgCrop , (wCal , imgSize))
            #centre image
            wGap = math.ceil((imgSize - wCal)/2)
            imgWhite[ : , wGap:wCal + wGap ] = imgResize
        

        else:
            k = imgSize/w
            hCal = math.ceil(k*h)
            imgResize = cv2.resize(imgCrop , (imgSize , hCal ))
            #centre image
            hGap = math.ceil((imgSize-hCal)/2)
            imgWhite[hGap:hCal + hGap, :] = imgResize
            
            
        
        # Display the cropped image only if it has valid dimensions
        if imgCrop.shape[0] > 0 and imgCrop.shape[1] > 0:
            cv2.imshow('ImageCrop', imgCrop)
        cv2.imshow('ImageWhite' , imgWhite)

        
    cv2.imshow('Image', img)
    key = cv2.waitKey(1)
    
    if key == ord("s"):
        counter += 1  
        cv2.imwrite( f'{folder}/Images_{time.time()}.jpg' ,imgWhite )
        print(counter)
    

cap.release()
cv2.destroyAllWindows()

In [ ]:
cap = cv2.VideoCapture(0)

while True:
    ret,frame = cap.read()
    
    roi = frame[100:400 , 320:620]
    cv2.imshow('roi' , roi)
    roi = cv2.cvtcolor(roi , cv2.COLOR_BGR2GRAY)
    roi = cv2.resize(roi,None, interpolation = cv2.INTER_AREA )
    
    cv2.imshow('roi scaled and grey',roi )
    copy = frame.copy()
    cv2.rectangle(copy , (320,100) , (620,400) , (255,0,0) , 5)
    
    roi = roi.reshape(1,28,28,1)
    
    result = str(model.predict_classes(roi,1,verbose = 0)[0])
    cv2.putText(copy,getLetter(result) , (300,100) , cv2.FONT_HERSHEY_COMPLEX , 2 , (0,255,0),2)
    
    cv2.imshow('frame',copy)
    
    if cv2.waitKey(1) == 13:
        break

cap.release()
cv2.destroyALLWindows()